### Prepare data

In [1]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/SiouxFalls_trips_simp.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                #perturb the demands slightly
                demands[(s,t)] = demand * 0.9
            end
        end
    end
end                
close(file)

include("load_network_uni-class.jl")

ta_data = load_ta_network("Sioux_simp");

In [2]:
using JuMP

function TAP(demands)
    #load OD pair-route incidence
    odPairRoute = readall("od_pair_route_incidence_Sioux_simp.json");
    odPairRoute = JSON.parse(odPairRoute);

    #load link-route incidence
    linkRoute = readall("link_route_incidence_Sioux_simp.json");
    linkRoute = JSON.parse(linkRoute);

    #load OD pair labels
    odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
    odPairLabel = JSON.parse(odPairLabel);

    odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
    odPairLabel_ = JSON.parse(odPairLabel_);

    #load link labels
    linkLabel = readall("link_label_dict_Sioux_simp.json");
    linkLabel = JSON.parse(linkLabel);

    linkLabel_ = readall("link_label_dict_Sioux_simp_.json");
    linkLabel_ = JSON.parse(linkLabel_);

    #load node-link incidence
    nodeLink = readall("node_link_incidence_Sioux_simp.json");
    nodeLink = JSON.parse(nodeLink);

    include("load_network_uni-class.jl")

    ta_data = load_ta_network("Sioux_simp");

    start_node = ta_data.start_node
    capacity = ta_data.capacity;
    free_flow_time = ta_data.free_flow_time;

    numNodes = maximum(map(pair->pair[1], keys(demands)));

    demandsVec = zeros(length(odPairLabel_))

    for i = 1:length(demandsVec)
        demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
    end

    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    numLinks = size(start_node)[1]
    numODpairs = numNodes * (numNodes - 1)

    @defVar(m, linkFlow[1:numLinks])

    @defVar(m, odLinkFlow[keys(demands), 1:numLinks])
    for (s,t) = keys(demands)
        for k = 1:numLinks
            @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
        end
    end

    for k = 1:numLinks
        linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
        @addConstraint(m, linkFlowPartial == linkFlow[k])
    end

    for l = 1:numNodes
        for (s,t) = keys(demands)
            if s != t
                odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
                if (l == s)
                    @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
                elseif (l == t)
                    @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
                else
                    @addConstraint(m, odLinkFlowPartial == 0)
                end
            end
        end
    end

    @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), 
        a = 1:numLinks})
#     @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^2)/(capacity[a]), 
#         a = 1:numLinks})

    @setNLObjective(m, Min, f)

    TT = STDOUT # save original STDOUT stream
    redirect_stdout()
    solve(m)
    redirect_stdout(TT) # restore STDOUT

    getValue(linkFlow)

    getObjectiveValue(m)

    outfile = open("flows_converge_simp.txt", "w")

    write(outfile, join(("From", "to", "Volume Capacity"), "        "), "\n")

    for i = 1:length(ta_data.start_node)
         n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], getValue(linkFlow)[i]
         write(outfile, join((n1, n2, n3), "        "), "\n")
    end

    close(outfile)
    
    return getValue(linkFlow), numNodes, numLinks, numODpairs
end

TAP (generic function with 1 method)

In [3]:
tapFlowVec, numNodes, numLinks, numODpairs = TAP(demands)

([2070.0,3510.0,2070.0,3150.0,3510.0,6240.91,2849.09,6284.98,9810.0,3900.91  …  7110.0,4410.0,12150.0,12150.0,7110.0,3854.98,7200.0,3885.02,2805.02,3929.09],12,30,132)

In [4]:
tapFlows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    tapFlows[key] = tapFlowVec[i]
end

In [5]:
tapFlows

Dict{(Int64,Int64),Float64} with 30 entries:
  (11,12) => 3885.0244956308134
  (4,3)   => 6284.975501544218
  (2,6)   => 3149.999998697045
  (11,4)  => 3854.9755015772
  (5,6)   => 7469.999998586949
  (4,11)  => 3900.905726908588
  (8,6)   => 7019.9999986354405
  (5,4)   => 9809.999998701325
  (10,11) => 7109.9999986643
  (2,1)   => 2069.999998584984
  (8,9)   => 4409.999998632529
  (12,3)  => 2805.0244956943825
  (11,10) => 7199.999998660668
  (3,1)   => 3509.999998610329
  (5,9)   => 7109.999998612249
  (6,2)   => 3149.9999986970824
  (6,8)   => 7019.99999863672
  (4,5)   => 9809.999998704987
  (1,3)   => 3509.999998610365
  (8,7)   => 5849.999998666154
  (7,8)   => 5849.999998666157
  (12,11) => 3929.094270295784
  (9,10)  => 12149.99999864232
  (3,12)  => 2849.0942703593632
  (6,5)   => 7469.999998585632
  ⋮       => ⋮

In [6]:
outfile = open("tapFlows.json", "w")

JSON.print(outfile, tapFlows)

close(outfile)

In [7]:
tapFlows = readall("tapFlows.json");
tapFlows = JSON.parse(tapFlows);